# CoffeeBeans Voice Agent - Interactive Testing

This notebook allows you to:
1. Test the conversation agent interactively
2. Test knowledge base functions
3. Analyze conversation state and sentiment

For architecture diagrams and system flow, see the main [README.md](../README.md)

## Setup and Imports

In [ ]:
import sys
import asyncio
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import agent components
from src.state import create_initial_state, ConversationState
from src.graph import (
    create_conversation_graph,
    supervisor_agent,
    update_state_from_transcript,
    analyze_sentiment,
    extract_interests,
    detect_objections,
    get_company_info,
    match_service_to_need,
    get_objection_response,
    schedule_next_step
)
from src.services import GroqService
from src.knowledge import knowledge_base
from src.config import settings

print("✅ Imports successful!")

In [ ]:
# Create the conversation graph
graph = create_conversation_graph()

print("Graph created successfully!")
print(f"Graph type: {type(graph)}")

## Test Knowledge Base Functions

In [ ]:
# Test service matching
print("\n🎯 Service Matching Examples:")
print("=" * 60)

pain_points = [
    "Our AI models won't scale to production",
    "We have serious data quality issues",
    "Need better security in our supply chain",
    "Legacy systems holding us back"
]

for pain_point in pain_points:
    result = match_service_to_need(pain_point)
    print(f"\n📌 Pain Point: '{pain_point}'")
    print(f"   Matched Service: {result['service_name']}")
    print(f"   Top Benefit: {result['benefits'][0]}")

In [ ]:
# Test objection handling
print("\n💬 Objection Handling Examples:")
print("=" * 60)

objections = ["too expensive", "not the right time", "we have an internal team"]

for objection in objections:
    response = get_objection_response(objection)
    print(f"\n❌ Objection: '{objection}'")
    print(f"   Response Framework:\n   {response['response_framework'][:150]}...")
    print(f"   Key Points: {', '.join(response['key_points'])}")

## Test Sentiment & Interest Analysis

In [ ]:
# Test sentiment analysis
print("😊 Sentiment Analysis:")
print("=" * 60)

test_inputs = [
    "Yes, I'm very interested in learning more about your AI solutions!",
    "We're struggling with getting our ML models into production.",
    "No thanks, I'm not interested right now.",
    "Maybe we can discuss this later."
]

for text in test_inputs:
    sentiment = analyze_sentiment(text)
    interests = extract_interests(text)
    objections = detect_objections(text)
    
    print(f"\nInput: '{text}'")
    print(f"  → Sentiment: {sentiment}")
    print(f"  → Interests: {interests if interests else 'None'}")
    print(f"  → Objections: {objections if objections else 'None'}")

## Interactive Agent Testing

In [ ]:
# Initialize the agent
class NotebookAgent:
    """Interactive agent for Jupyter notebook testing."""
    
    def __init__(self):
        self.llm = GroqService()
        self.state = create_initial_state("NOTEBOOK-TEST", "NB-SESSION-001")
        self.conversation_history = []
        
        # Initialize with supervisor prompt
        self.state = supervisor_agent(self.state)
        
    async def send_message(self, user_message: str) -> str:
        """Send a message and get agent response."""
        # Update state with user input
        self.state = update_state_from_transcript(self.state, user_message, "user")
        
        # Get system instruction from state
        messages = self.state.get("messages", [])
        system_instruction = None
        if messages and messages[-1]["role"] == "system":
            system_instruction = messages[-1]["content"]
        
        # Get LLM response
        response = await self.llm.get_response(user_message, system_instruction)
        
        return response
    
    def show_status(self):
        """Display current conversation state."""
        print("\n" + "=" * 60)
        print("📊 CONVERSATION STATUS")
        print("=" * 60)
        print(f"Phase: {self.state.get('phase', 'unknown').upper()}")
        print(f"Sentiment: {self.state.get('sentiment', 'neutral')}")
        print(f"Engagement: {self.state.get('engagement_level', 'medium')}")
        print(f"Interests: {', '.join(self.state.get('interests', [])) or 'None'}")
        print(f"Objections: {', '.join(self.state.get('objections_raised', [])) or 'None'}")
        print(f"Messages: {len(self.llm.conversation_history)}")
        print("=" * 60)

# Create agent instance
agent = NotebookAgent()
print("✅ Agent initialized and ready!")

In [ ]:
# Start conversation - Get initial greeting
print("🎙️ CoffeeBeans Voice Agent - Interactive Session")
print("=" * 60)

greeting = await agent.send_message("")
print(f"\n🤖 AGENT: {greeting}")

In [ ]:
# You can chat with the agent by running this cell multiple times
# Modify the user_input variable each time

user_input = "Yes, I have a few minutes. We're actually struggling with our AI deployment."

print(f"👤 YOU: {user_input}")
response = await agent.send_message(user_input)
print(f"\n🤖 AGENT: {response}")

In [ ]:
# Continue conversation
user_input = "We have built several ML models but can't get them to production scale."

print(f"👤 YOU: {user_input}")
response = await agent.send_message(user_input)
print(f"\n🤖 AGENT: {response}")

In [ ]:
# Check conversation status
agent.show_status()

In [ ]:
# Test objection handling
user_input = "This sounds interesting, but I'm worried about the cost. What kind of budget are we talking about?"

print(f"👤 YOU: {user_input}")
response = await agent.send_message(user_input)
print(f"\n🤖 AGENT: {response}")

In [ ]:
# Check updated status
agent.show_status()

## Full Conversation Example

In [ ]:
# Run a complete conversation scenario
async def run_conversation_scenario():
    """Run a predefined conversation scenario."""
    
    # Create new agent
    scenario_agent = NotebookAgent()
    
    conversation = [
        "",  # Initial greeting
        "Yes, I have a couple of minutes.",
        "We're a fintech startup struggling with data pipeline issues and AI integration.",
        "Our data quality is poor and we can't trust our analytics.",
        "That sounds relevant. How have you helped other fintech companies?",
        "Interesting. But we already have an internal data team.",
        "I'd like to learn more. Can you send me some information?"
    ]
    
    print("🎬 Running Full Conversation Scenario")
    print("=" * 70)
    
    for i, user_msg in enumerate(conversation):
        if user_msg:
            print(f"\n👤 USER: {user_msg}")
        
        response = await scenario_agent.send_message(user_msg)
        print(f"🤖 AGENT: {response}")
        print("-" * 70)
    
    print("\n📊 Final Conversation Analysis:")
    scenario_agent.show_status()

# Run the scenario
await run_conversation_scenario()

## Conversation Analytics

In [ ]:
# Analyze conversation patterns
import matplotlib.pyplot as plt
import pandas as pd

# Sample data from multiple test conversations
test_conversations = [
    {"sentiment": "positive", "interests": ["ai", "data"], "objections": [], "outcome": "scheduled_call"},
    {"sentiment": "neutral", "interests": ["blockchain"], "objections": ["cost"], "outcome": "send_info"},
    {"sentiment": "negative", "interests": [], "objections": ["timing", "need_info"], "outcome": "not_interested"},
    {"sentiment": "positive", "interests": ["ai", "cloud"], "objections": ["internal_team"], "outcome": "scheduled_call"},
    {"sentiment": "neutral", "interests": ["data", "legacy"], "objections": [], "outcome": "send_info"},
]

# Create visualizations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Sentiment distribution
sentiments = [conv["sentiment"] for conv in test_conversations]
sentiment_counts = pd.Series(sentiments).value_counts()
axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=['green', 'orange', 'red'])
axes[0].set_title('Sentiment Distribution')
axes[0].set_ylabel('Count')

# Outcomes distribution
outcomes = [conv["outcome"] for conv in test_conversations]
outcome_counts = pd.Series(outcomes).value_counts()
axes[1].bar(outcome_counts.index, outcome_counts.values, color=['skyblue', 'lightgreen', 'salmon'])
axes[1].set_title('Conversation Outcomes')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Interest areas
all_interests = []
for conv in test_conversations:
    all_interests.extend(conv["interests"])
interest_counts = pd.Series(all_interests).value_counts()
axes[2].bar(interest_counts.index, interest_counts.values, color='purple', alpha=0.6)
axes[2].set_title('Top Interest Areas')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

print("\n📈 Conversation Analytics Summary:")
print(f"  • Total Conversations: {len(test_conversations)}")
print(f"  • Positive Sentiment: {sentiments.count('positive')/len(sentiments)*100:.1f}%")
print(f"  • Successful Outcomes: {(outcomes.count('scheduled_call')+outcomes.count('send_info'))/len(outcomes)*100:.1f}%")
print(f"  • Most Common Interest: {interest_counts.index[0] if len(interest_counts) > 0 else 'None'}")

## Reset Agent

Run this cell to start a fresh conversation

In [ ]:
# Reset the agent for a new conversation
agent = NotebookAgent()
print("✅ Agent reset! Ready for a new conversation.")